### Predicting drug response using the GDSC model

This example shows how to process and predict drug response using the GDSC model based on scRNA-seq data. 

1. Read normalized gene expression file (log2 fold-change generated in B01) of cell clusters
2. Calculate kernel features
3. Predicting drug response using pre-trained GDSC model

In [1]:
import sys, os, pickle
import pandas as pd
import importlib

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy import stats

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
mpl.rc("savefig", dpi=300)

scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))

from cadrres import pp, model, evaluation, utility

### Read gene expression file and calculate kernel features

##### Indicate input files and output directory

In [2]:
cluster_norm_exp_fname = '../preprocessed_data/HN_patient_specific/log2_fc_cluster_tmm.csv'
output_dir = '../result/HN_model/'

In [3]:
cluster_norm_exp_df = pd.read_csv(cluster_norm_exp_fname, index_col=0).T
cluster_norm_exp_df.head()

cluster,A1,A2,B1,B2,C1,C2,D1,D2,E1,E2,F1,F2,F3,G1,H1,H2,I1,I2,J1,K
AAAS,0.846317,0.377929,-0.730985,0.734532,0.082111,0.487773,-0.157648,-0.895361,1.158548,0.186290,-0.467435,-1.184772,-0.512514,1.137256,-0.135901,-0.411667,-0.364310,-0.473204,-0.587701,0.910743
AAMP,1.056943,-0.036515,-0.257559,-0.098359,0.459542,0.703635,-1.245455,-0.695861,0.327241,0.225542,0.908263,0.745428,0.783679,0.988684,-0.559639,-0.745485,-0.508481,-1.365684,-1.828199,1.142280
AARS,-0.035398,0.437952,-0.265018,0.651783,-0.108906,0.812684,-1.497611,-1.530190,1.863446,1.355937,0.348401,0.524545,0.722933,1.243781,-0.362966,-0.985351,-1.312736,-1.792299,-0.685442,0.614455
AARS2,0.618334,0.373205,-0.413254,-0.088507,-0.753614,-0.384261,0.289013,0.018347,-0.065066,-0.490258,-0.705108,0.242078,0.017760,0.054583,-0.064942,0.047676,0.164926,0.414661,-0.501854,1.226283
AASDHPPT,0.943819,-0.337211,1.537949,1.188424,-1.104427,-1.000841,-0.361750,-1.377895,-0.751011,-0.819170,0.960921,0.622617,0.237355,0.616813,-0.121516,-0.351838,-0.511488,-0.424887,0.060427,0.993710


##### Read GDSC gene expression

In [4]:
gdsc_log2_exp_df = pd.read_csv('../data/GDSC/GDSC_exp.tsv', sep='\t', index_col=0)
gdsc_norm_exp_df, gdsc_mean_exp_df = pp.gexp.normalize_log2_mean_fc(gdsc_log2_exp_df)
gdsc_norm_exp_df.head()

,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.941884,0.858532,2.022198,1.107003,1.039128,0.384394,-3.404941,0.271467,-0.747093,-3.234189,...,0.415498,-3.453636,-3.651247,1.683084,0.242039,1.751489,1.732782,1.399116,-3.577806,0.462988
TNMD,0.108897,-0.077972,-0.212181,-0.037766,0.102051,0.033988,-0.027485,0.019062,-0.168814,0.434495,...,-0.056842,-0.110552,0.120717,-0.003137,-0.233059,-0.216412,0.024201,-0.334520,0.014779,-0.021403
DPM1,0.343465,1.771254,-0.155355,-0.152617,0.382752,-0.262101,0.228297,0.169844,0.263669,1.534067,...,0.450398,0.406864,0.275874,0.418742,0.382388,1.427654,0.521690,0.756662,-0.162186,0.752131
SCYL3,-0.632019,-0.179926,-0.290583,-0.183111,0.094688,0.024090,1.721355,-0.531780,-0.398700,1.314070,...,-0.549978,0.377200,0.101711,-0.388691,-0.299252,0.179037,-0.696423,0.196524,0.020015,-0.146319
C1orf112,-0.313637,0.038166,-0.457698,-0.135904,0.146055,0.120737,-0.682451,-0.425870,-0.341484,-0.123090,...,0.032515,0.253426,0.112265,-0.497331,0.120512,0.690414,0.552870,-0.622959,-0.464122,-0.258524


##### Read an essential gene list

In [5]:
ess_gene_list = utility.get_gene_list('../data/essential_genes.txt')
selected_gene_list = [g for g in ess_gene_list if g in cluster_norm_exp_df.index]
len(selected_gene_list)

1729

##### Calculate kernal features

In [6]:
cluster_kernel_df = pp.gexp.calculate_kernel_feature(cluster_norm_exp_df, gdsc_norm_exp_df, selected_gene_list)

Calculating kernel features based on 1547 common genes
(1729, 20) (17737, 1018)


In [7]:
cluster_kernel_df.to_csv(output_dir + "cluster_kernel_df.csv")

### Predicting drug response using pre-trained GDSC model

In [8]:
importlib.reload(pp)
importlib.reload(model)
importlib.reload(evaluation)
importlib.reload(utility)

<module 'cadrres.utility' from '/Users/Nok/Dropbox/Research/2019_drug_response_heterogeneity/CaDRReS_package/cadrres/utility.py'>

In [9]:
model_name = 'hn_drug_cw_dw10_100000'

In [10]:
model_dict = pickle.load(open('../result/HN_model/{}_param_dict.pickle'.format(model_name), 'rb'))
model_training_output_dict = pickle.load(open('../result/HN_model/{}_output_dict.pickle'.format(model_name), 'rb'))

Make a prediction

In [11]:
pred_df, P_df = model.predict_from_model(model_dict, cluster_kernel_df)
pred_df.to_csv(os.path.join(output_dir,'pred_{}_model.csv'.format('gdsc_no_bias_{}').format(model_name)))
P_df.to_csv(os.path.join(output_dir,'P_{}_model.csv'.format('gdsc_no_bias_{}').format(model_name)))